# Q1) Construction with Constraints

In [1]:
# Decision Variables 
# -- In each month, how many workers do you assign to each project
# Variables of how many people working for each 3 project in every of 4 months -- 12 all together

# Costraints 
# -- Only 6 workers can work on a single job
# -- 8/10/12 worker hours required for projects 1/2/3
# -- 3/4/2 months max complete time for projects 1/2/3

# Objective 
# -- Minimize time spent finishing projects

In [2]:
using JuMP, HiGHS

In [3]:
model = Model(HiGHS.Optimizer)

# x is workers at each project per month
# and 6 workers on single job at a time
@variable(model, 0 <= x[i = 1:3, j = 1:4] <= 6)

for j in 1:4
    # 8 total workers
    @constraint(model, sum( x[:, j]) == 8) 
end

# Worker hours required to complete each project 1:3
@constraint(model, sum(x[1, :]) >= 8)
@constraint(model, sum(x[2, :]) >= 10)
@constraint(model, sum(x[3, :]) >= 12)

# Maximum complete times in time frame
@constraint(model, x[1, 4] == 0)
@constraint(model, x[3, 3] == 0)
@constraint(model, x[3, 4] == 0)

# If you don't specify an objective, it'll just find a feasible point
# If there's no point it'll print out a lot of junk
# Check termination status to look for if it found something
optimize!(model)

println("\n", termination_status(model))

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No invertible representation for getDualRay

INFEASIBLE


The three projects cannot all be completed on time!

# Q2) Stigler's Diet

## Stigler Diet Problem: Starter Code

In [4]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("NamedArrays")


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [5]:
# STARTER CODE FOR STIGLER'S DIET PROBLEM

using NamedArrays, CSV, DataFrames

# import Stigler's data set
raw = CSV.read("stigler.csv", DataFrame);
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 3:m          # rows containing food names

# list of food
foods = raw[2:end,1]
# list of nutrients
nutrients = [string(names(raw)[i]) for i=2:length(names(raw))]

# minimum required amount of each nutrient
lower = Dict( zip(nutrients,raw[1,n_nutrients]) )

# data[f,i] is the amount of nutrient i contained in food f 
dataraw = Matrix(values(raw[2:end,2:end]))
data = NamedArray(dataraw,(foods,nutrients),("foods","nutrients"))

        
println("Foods:\n")
for i in foods
    println(i)
end


println("\n\nNutrient Lower Bounds:\n")
for j in nutrients
    println(j," at least: ",lower[j])
end


Foods:

Wheat Flour (Enriched)
Macaroni
Wheat Cereal (Enriched)
Corn Flakes
Corn Meal
Hominy Grits
Rice
Rolled Oats
White Bread (Enriched)
Whole Wheat Bread
Rye Bread
Pound Cake
Soda Crackers
Milk
Evaporated Milk (can)
Butter
Oleomargarine
Eggs
Cheese (Cheddar)
Cream
Peanut Butter
Mayonnaise
Crisco
Lard
Sirloin Steak
Round Steak
Rib Roast
Chuck Roast
Plate
Liver (Beef)
Leg of Lamb
Lamb Chops (Rib)
Pork Chops
Pork Loin Roast
Bacon
Ham, smoked
Salt Pork
Roasting Chicken
Veal Cutlets
Salmon, Pink (can)
Apples
Bananas
Lemons
Oranges
Green Beans
Cabbage
Carrots
Celery
Lettuce
Onions
Potatoes
Spinach
Sweet Potatoes
Peaches (can)
Pears (can)
Pineapple (can)
Asparagus (can)
Green Beans (can)
Pork and Beans (can)
Corn (can)
Peas (can)
Tomatoes (can)
Tomato Soup (can)
Peaches, Dried
Prunes, Dried
Raisins, Dried
Peas, Dried
Lima Beans, Dried
Navy Beans, Dried
Coffee
Tea
Cocoa
Chocolate
Sugar
Corn Syrup
Molasses
Strawberry Preserves


Nutrient Lower Bounds:

Calories (1000) at least: 3.0
Protein (

### Part A)

In [6]:
# Minimize cost while allowing the daily recommended amount of several nutrients listed above

# The decision variables are how many dollars are spent on each food type 
# -- i.e. how many multiples of the nutrients listed in each row you will receive
# -- There will be one decision variable per food type
# --> The dollars spent on each food type have to be >= 0 since you can't spend negative money

# The constraints are the total number of nutrients needing to be above what is shown there
# -- Calories >= 3 (thousand)
# -- Protein >= 70 (grams)
# -- Calcium >= 0.8 (grams)
# -- Iron >= 12 (mg) 
# -- Vitamin A >= 5.0 (1000 IU)
# -- Thiamine >= 1.8(mg)
# -- Riboflavin >= 2.7 (mg)
# -- Niacin >= 18 (mg)
# -- Ascorbic Acid >= 75 (mg)
# --> We find the total number of nutrients we end up with by for each nutrient, summing across 
#     the product of the amount of money spent on each food and the data table entry of nutrients per $1

# The objective is to minimize the cost spent across all foods

### Part B)

In [7]:
model = Model(HiGHS.Optimizer)

# Amount spent on each food
@variable(model, foodmoney[f in foods] >= 0)

# Nutrients needed
@constraint(model, nutneeds[n in nutrients], sum(foodmoney[f]*data[f,n] for f in foods) >= lower[n])

# Minimize cost
@objective(model, Min, sum(foodmoney[f] for f in foods))

optimize!(model)

# Print out results
println()
for f in foods
    if value(foodmoney[f]) != 0.0
        println("We buy ", value(foodmoney[f]), " units of ", f)
    end
end

println()
println("Annual cost: ", 365*sum(value(foodmoney[f]) for f in foods) )

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
9 rows, 40 cols, 317 nonzeros
9 rows, 27 cols, 213 nonzeros
Presolve : Reductions: rows 9(-0); columns 27(-50); elements 213(-357)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 9(76.4375) 0s
          5     1.0866227821e-01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  1.0866227821e-01
HiGHS run time      :          0.00

We buy 0.02951906167648827 units of Wheat Flour (Enriched)
We buy 0.001892557290705264 units of Liver (Beef)
We buy 0.01121443524614487 units of Cabbage
We buy 0.0050076604667252025 units of Spinach
We buy 0.06102856352669324 units of Navy Beans, Dried

Annual cost: 39.66173154546625


In [8]:
# This diet is about 27 cents cheaper than Stigler's diet

### Part C) Print lower bounds and then print actual we have

In [9]:
for n in nutrients
    println( "The lower bound for ", n, " is: ", lower[n], " and the optimal amount is: ", value(nutneeds[n]) )
end

The lower bound for Calories (1000) is: 3.0 and the optimal amount is: 3.0
The lower bound for Protein (g) is: 70 and the optimal amount is: 147.41353494220905
The lower bound for Calcium (g) is: 0.8 and the optimal amount is: 0.8
The lower bound for Iron (mg) is: 12 and the optimal amount is: 60.466922101736586
The lower bound for Vitamin A (1000 IU) is: 5.0 and the optimal amount is: 5.0
The lower bound for Thiamine (mg) is: 1.8 and the optimal amount is: 4.120438804838622
The lower bound for Riboflavin (mg) is: 2.7 and the optimal amount is: 2.7
The lower bound for Niacin (mg) is: 18 and the optimal amount is: 27.31598070028832
The lower bound for Ascorbic Acid (mg) is: 75 and the optimal amount is: 75.0


In [10]:
# Calories, Calcium, Vitamin A, Riboflavin, and Ascordbic Acid are all active 
# --> That is, their constraints are inequalities and yet for them the left and right hand sides are both equal
#    This is shown above for example in Calories where the lower bound is 3.0 and the optimal amount is 3.0
#    This means that the dual variable that corresponds to this constraint is allowed to be nonzero whereas 
#    The other dual variables will have to be 0
# Complementarity is satisfied

In [13]:
# Verifying this by looking at the dual variables and seeing which are 0...
#    All zero dual variables were inactive
#    All nonzero dual variables were active
for n in nutrients
    println( "The dual variable for ", n, " is: ", dual(nutneeds[n]) )
end


The dual variable for Calories (1000) is: 0.0056890897818463754
The dual variable for Protein (g) is: 0.0
The dual variable for Calcium (g) is: 0.02983996414269529
The dual variable for Iron (mg) is: 0.0
The dual variable for Vitamin A (1000 IU) is: 0.000341883335328549
The dual variable for Thiamine (mg) is: 0.0
The dual variable for Riboflavin (mg) is: 0.020601133887870167
The dual variable for Niacin (mg) is: 0.0
The dual variable for Ascorbic Acid (mg) is: 0.00014354290323833802


### Part D) New optimization Problem

In [14]:
model = Model(HiGHS.Optimizer)

# Amount spent on each food
@variable(model, foodmoney[f in foods] >= 0)

# Nutrients needed
@constraint(model, nutneeds[n in nutrients], sum(foodmoney[f]*data[f,n] for f in foods) >= lower[n])
@constraint(model, foodmoney["Liver (Beef)"] == 0)
@constraint(model, foodmoney["Milk"] >= .01)

# Minimize cost
@objective(model, Min, sum(foodmoney[f] for f in foods))

optimize!(model)

# Print out results
println()
for f in foods
    if value(foodmoney[f]) != 0.0
        println("We buy ", value(foodmoney[f]), " units of ", f)
    end
end

println()
println("Annual cost: ", 365*sum(value(foodmoney[f]) for f in foods) )

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
9 rows, 40 cols, 315 nonzeros
9 rows, 28 cols, 220 nonzeros
Presolve : Reductions: rows 9(-2); columns 28(-49); elements 220(-352)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     1.0000006980e-02 Pr: 9(75.9634) 0s
          5     1.1194951710e-01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  1.1194951710e-01
HiGHS run time      :          0.00

We buy 0.03569701410638522 units of Wheat Flour (Enriched)
We buy 0.01 units of Milk
We buy 0.0021523432962831084 units of Evaporated Milk (can)
We buy 0.010991091937036563 units of Cabbage
We buy 0.005114223881043092 units of Spinach
We buy 0.047994843882913016 units of Navy Beans, Dried

Annual cost: 40.86157374283626
